In [1]:
import pandas as pd
from scipy.sparse import coo_matrix
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Input, Embedding, LSTM, Dense, Dot, Softmax, Concatenate, BatchNormalization, Attention, \
    Flatten
from keras.models import Model
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
import numpy as np
from keras.utils import pad_sequences
pd.set_option('display.max_colwidth', None)
from keras.layers import Input, Embedding, LSTM, Dense, Dropout, Attention
from keras.models import Model
from sklearn.preprocessing import LabelEncoder

In [2]:
#%pip install tensorflow
#%pip install tensorflow-metal
#%pip uninstall tensorflow --yes
#%pip uninstall tensorflow-metal --yes

In [3]:
if tf.config.list_physical_devices('GPU'):
    print("GPU is available")
else:
    print("GPU is not available")

GPU is available


In [4]:
basepath = '../../datasets/preprocessed_datasets/gabor/'
interaction_data = pd.read_pickle(basepath + 'user_item_interactions_only_main_products.pkl')
# interaction_data = interaction_data.sample(n=10000)
interaction_data


,customer_id,main_product_id,amount
0,1940761,3696391,1
1,1940761,3850720,1
2,1940761,4839681,2
3,1940761,5015362,2
4,1940761,6523930,1
...,...,...,...
585121,11128024,3711313,1
585122,11128039,10224959,1
585123,11128082,4914111,1
585124,11128095,10451987,1


In [5]:
product_data = pd.read_pickle(basepath + 'no_product_variants_with_duplicate_ids.pkl')
display(product_data)

,main_product_id,productNumber,productName,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,line,...,heelHeightGroup,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor,duplicate_product__ids,product_sizes
0,557559,4058394021466,sportliche Ballerinas Glattleder schwarz,schwarz,02.643.57,6590678,315571,w,PT,F-S|H-W,...,bis 3 cm,40.0,6.5,11.0,0.0,Gummi,0.0,0.0,"[557559, 549733, 549734, 557553, 589488, 589480, 589481, 589486, 549735, 589489, 589477, 589490, 589483, 589479, 557556, 589485, 589482, 589478, 589475, 549732, 549729, 549730, 589484, 549738, 549731, 557561, 589487]","[35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39.0, 40.0, 40.5, 41.0, 42.0, 42.5, 43.0, 44.0, 44.5, 46.0]"
1,551622,4054452768212,elegante Pumps Glattleder schwarz,schwarz,05.160.37,6590677,315573,w,PT,F-S|H-W,...,3 cm - 5 cm,38.0,5.0,8.0,0.0,EVA,0.0,0.0,"[551622, 551505, 551510, 551509, 552973, 551621, 551501, 551503, 551502, 556960, 551628, 552970, 556954, 556964, 552966, 551500, 551629, 604774, 604773, 604772, 551508, 604776, 552972, 551617, 552969, 604771, 556958, 604778, 551630, 556952, 551620, 551627, 604782, 604775, 551504, 551506, 551619, 604769, 552975, 552974, 604768, 551507, 552971, 556956, 551623, 551511, 551626, 604780, 552964, 604779, 551631, 551624, 556953, 604777, 551512, 552968, 552963, 552962, 552976, 604770, 552967, 556951, 556961, 556955, 556950]","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39.0, 40.0, 40.5, 41.0, 42.0, 42.5, 43.0, 44.0, 44.5]"
2,547193,4054452768427,Slipper Glattleder schwarz,schwarz,04.443.27,6590677,315576,w,SK,F-S|H-W,...,3 cm - 5 cm,42.0,8.0,14.0,0.0,PU-TPU,0.0,0.0,"[547193, 547191, 547196, 547187, 547189, 547195, 547192, 547190, 547185, 547188, 547182, 547183, 547184, 547194, 547186, 3483121, 3483123, 3483120, 3554723, 3554724, 3554732, 3554725, 3554729, 3483124, 3483129, 3591864, 3591866, 3554731, 3483128, 3694614, 3694623, 3694616, 3554730, 3711336, 3483126, 3554733, 3694625, 3483133, 3711328, 3711335, 3592196, 3711332, 3554726, 3743383, 3592206, 3743385, 3694615, 3483127, 3694613, 3711737, 3743393, 3743384, 3591852, 3711337, 3592197, 3591857, 3591859, 3743382, 3743390, 3591865, 3483122, 3743388, 3591855, 3743387, 3483134, 3592199, 3592202, 3483125, 3694618, 3591860, 3743392, 3743389, 3592208, 3592198, 3483131, 3591853, 3694621, 3592207, 3592203, 3711330, 3554734, 3711334, 3743386, 3554728, 3694617, 4030773, 3694622, 3743391, 3711331, 3711329, 4093389, 3694612, 4093391, 4093397, 4030774, 3711324, 4030778, 4030777, 4093398, 4030776, ...]","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39.0, 40.0, 40.5, 41.0, 42.0, 42.5, 43.0, 44.0]"
3,549776,4059701687894,Sneaker low Rauleder blau,blau,06.968.46,4997827,315567,w,VN,F-S|H-W,...,3 cm - 5 cm,39.0,6.0,10.0,0.0,Gummi-EVA,0.0,0.0,"[549776, 550599, 550596, 550593, 550601, 549780, 549778, 549779, 550594, 550595, 549781, 550597, 550598, 549775, 550602, 549782, 550588, 549777, 549783, 549774, 549773, 550592, 550600, 550589, 550604, 549770, 549771, 550591, 3478987, 3505296, 3505294, 3479430, 3505289, 3479436, 3478986, 3555129, 3479431, 3479432, 3505288, 3591679, 3478985, 3591667, 3505293, 3479429, 3555125, 3591672, 3479434, 3591798, 3555126, 3591674, 3555130, 3591675, 3591668, 3555124, 3591797, 3591669, 3591803, 3591680, 3591681, 3478990, 3591793, 3591799, 3591796, 3505775, 3505292, 3504385, 3505768, 3479433, 3711254, 3479437, 3591673, 3711249, 3505771, 3555123, 3505769, 3505772, 3711248, 3478988, 3505776, 3505778, 3479435, 3692987, 3711244, 3711251, 3711257, 3711258, 3787796, 3479424, 3787792, 3845553, 3845549, 3845547, 3845546, 3847557, 3787799, 3847554, 3591671, 3479425, 3847731, 3591790, ...]","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39.0, 40.0, 40.5, 41.0, 42.0, 42.5, 43.0, 44.0, 44.5, 46.0]"
4,550726,4054452851594,elegante Ballerinas Materialmix Lederimitat schwarz,schwarz,06.102.67,6590678,315574,w,SK,F-S|H-W,...,bis 3 cm,37.5,4.5,7.0,0.0,EVA,0.0,0.0,"[550726, 550736, 550735, 550732, 550728, 550722, 550729, 550731, 550727, 550

In [6]:
product_data = product_data[['main_product_id', 'productName', 'gender', 'line', 'b2cPrice', 'duplicate_product__ids']]
product_data

,main_product_id,productName,gender,line,b2cPrice,duplicate_product__ids
0,557559,sportliche Ballerinas Glattleder schwarz,w,F-S|H-W,99.95,"[557559, 549733, 549734, 557553, 589488, 589480, 589481, 589486, 549735, 589489, 589477, 589490, 589483, 589479, 557556, 589485, 589482, 589478, 589475, 549732, 549729, 549730, 589484, 549738, 549731, 557561, 589487]"
1,551622,elegante Pumps Glattleder schwarz,w,F-S|H-W,89.95,"[551622, 551505, 551510, 551509, 552973, 551621, 551501, 551503, 551502, 556960, 551628, 552970, 556954, 556964, 552966, 551500, 551629, 604774, 604773, 604772, 551508, 604776, 552972, 551617, 552969, 604771, 556958, 604778, 551630, 556952, 551620, 551627, 604782, 604775, 551504, 551506, 551619, 604769, 552975, 552974, 604768, 551507, 552971, 556956, 551623, 551511, 551626, 604780, 552964, 604779, 551631, 551624, 556953, 604777, 551512, 552968, 552963, 552962, 552976, 604770, 552967, 556951, 556961, 556955, 556950]"
2,547193,Slipper Glattleder schwarz,w,F-S|H-W,79.95,"[547193, 547191, 547196, 547187, 547189, 547195, 547192, 547190, 547185, 547188, 547182, 547183, 547184, 547194, 547186, 3483121, 3483123, 3483120, 3554723, 3554724, 3554732, 3554725, 3554729, 3483124, 3483129, 3591864, 3591866, 3554731, 3483128, 3694614, 3694623, 3694616, 3554730, 3711336, 3483126, 3554733, 3694625, 3483133, 3711328, 3711335, 3592196, 3711332, 3554726, 3743383, 3592206, 3743385, 3694615, 3483127, 3694613, 3711737, 3743393, 3743384, 3591852, 3711337, 3592197, 3591857, 3591859, 3743382, 3743390, 3591865, 3483122, 3743388, 3591855, 3743387, 3483134, 3592199, 3592202, 3483125, 3694618, 3591860, 3743392, 3743389, 3592208, 3592198, 3483131, 3591853, 3694621, 3592207, 3592203, 3711330, 3554734, 3711334, 3743386, 3554728, 3694617, 4030773, 3694622, 3743391, 3711331, 3711329, 4093389, 3694612, 4093391, 4093397, 4030774, 3711324, 4030778, 4030777, 4093398, 4030776, ...]"
3,549776,Sneaker low Rauleder blau,w,F-S|H-W,140.00,"[549776, 550599, 550596, 550593, 550601, 549780, 549778, 549779, 550594, 550595, 549781, 550597, 550598, 549775, 550602, 549782, 550588, 549777, 549783, 549774, 549773, 550592, 550600, 550589, 550604, 549770, 549771, 550591, 3478987, 3505296, 3505294, 3479430, 3505289, 3479436, 3478986, 3555129, 3479431, 3479432, 3505288, 3591679, 3478985, 3591667, 3505293, 3479429, 3555125, 3591672, 3479434, 3591798, 3555126, 3591674, 3555130, 3591675, 3591668, 3555124, 3591797, 3591669, 3591803, 3591680, 3591681, 3478990, 3591793, 3591799, 3591796, 3505775, 3505292, 3504385, 3505768, 3479433, 3711254, 3479437, 3591673, 3711249, 3505771, 3555123, 3505769, 3505772, 3711248, 3478988, 3505776, 3505778, 3479435, 3692987, 3711244, 3711251, 3711257, 3711258, 3787796, 3479424, 3787792, 3845553, 3845549, 3845547, 3845546, 3847557, 3787799, 3847554, 3591671, 3479425, 3847731, 3591790, ...]"
4,550726,elegante Ballerinas Materialmix Lederimitat schwarz,w,F-S|H-W,79.95,"[550726, 550736, 550735, 550732, 550728, 550722, 550729, 550731, 550727, 550733, 550734, 550730, 550725]"
...,...,...,...,...,...,...
3292,10603553,Sandale met plateauzool Suède blauw,w,F-S,99.95,"[10603553, 10544662]"
3293,10442948,Mokassin Materialmix Leder pink,w,F-S,89.95,"[10442948, 10442949]"
3294,10782704,Shopper ANDIE blau,w,F-S,89.99,[10782704]
3295,10679703,Elegante pumps Glad leer wit,w,F-S,99.95,[10679703]


In [15]:


encoder = LabelEncoder()
interaction_data['customer_id_encoded'] = encoder.fit_transform(interaction_data['customer_id'])
interaction_data['product_id_encoded'] = encoder.fit_transform(interaction_data['main_product_id'])

# Create a user-item matrix
user_item_matrix = coo_matrix((interaction_data['amount'], (interaction_data['customer_id_encoded'], interaction_data['product_id_encoded'])))
# user_item_matrix = user_item_matrix.tocsr()
user_item_matrix_dense = user_item_matrix.toarray()
# Calculate the cosine similarity between products

#Truncated Backpropagation
timesteps = 1000
shape = (user_item_matrix_dense.shape[0] - timesteps + 1, timesteps, user_item_matrix_dense.shape[1])
strides = (user_item_matrix_dense.strides[0],) + user_item_matrix_dense.strides
X = np.lib.stride_tricks.as_strided(user_item_matrix_dense, shape=shape, strides=strides)
y = user_item_matrix_dense[timesteps-1:]

In [16]:
display(X.shape, y.shape)

(356780, 1000, 3297)

(356780, 3297)

In [17]:
# define the input shape
input_layer = Input(shape=(timesteps, user_item_matrix_dense.shape[1]))
user_item_matrix_dense = user_item_matrix.toarray()
# define the embedding layer
# embedding_layer = Embedding(input_dim=user_item_matrix.shape[1], output_dim=32)(input_layer)
# display(embedding_layer)
# define the LSTM layer
lstm_layer = LSTM(units=32, return_sequences=True)(input_layer)
# define the attention layer
attention_layer = Attention()([lstm_layer, lstm_layer])
# define the dense layer
flatten_layer = Flatten()(attention_layer)
# define the dense layer
dense_layer = Dense(units=1)(flatten_layer)

# define the output layer
output_layer = Dense(units=user_item_matrix.shape[1], activation='softmax')(dense_layer)
# create the model
model = Model(inputs=input_layer, outputs=output_layer)

In [18]:
# compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse', 'mae', 'accuracy'])
#model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse', 'mae', 'accuracy'])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 1000, 3297)]         0         []                            
                                                                                                  
 lstm_1 (LSTM)               (None, 1000, 32)             426240    ['input_2[0][0]']             
                                                                                                  
 attention_1 (Attention)     (None, 1000, 32)             0         ['lstm_1[0][0]',              
                                                                     'lstm_1[0][0]']              
                                                                                                  
 flatten_1 (Flatten)         (None, 32000)                0         ['attention_1[0][0]']   

In [ ]:
history = model.fit(X, y, epochs=10)

In [ ]:
history.history

In [ ]:
test = model.evaluate(user_item_matrix_dense, user_item_matrix_dense)
test

In [ ]:
loss, mse, mae, acc = model.evaluate(user_item_matrix_dense, user_item_matrix_dense)
print("Test loss: {:.4f}, Test MSE: {:.4f}".format(loss, mse))
print("Test MAE: {:.4f}, Test Accuracy: {:.4f}".format(mae, acc))